Question 1:
write the output inside the readme????

In [3]:
#function 1
def temp_tester(normal_temp):    
    
    def embedded_tester(test_temp):
      if abs(normal_temp - test_temp) <=1.0:
        return (True)
      else:
        return(False)
  
    return embedded_tester

In [5]:
human_tester = temp_tester(37)
chicken_tester = temp_tester(41.1)

chicken_tester(42) # True -- i.e. not a fever for a chicken


True

In [6]:
human_tester(42)   # False -- this would be a severe fever for a human


False

In [7]:
chicken_tester(43) # False

False

In [8]:
human_tester(35)   # False -- too low

False

In [9]:
human_tester(98.6) # False -- normal in degrees F but our reference temp was in degrees C

False

Question 2: